In [33]:
import pandas as pd

first_buildings = pd.read_csv("sql/first_buildings.csv")
print(first_buildings)

       gametime         platformgameid  teamid buildingtype
0        710694  ESPORTSTMNT01:1431635     100       turret
1       1863543  ESPORTSTMNT03:2073826     100    inhibitor
2        655449  ESPORTSTMNT02:1331126     200       turret
3       1458069  ESPORTSTMNT03:2542608     200    inhibitor
4       1018478  ESPORTSTMNT05:2942358     100       turret
...         ...                    ...     ...          ...
74992   1440023  ESPORTSTMNT05:1870222     100    inhibitor
74993   2349105  ESPORTSTMNT01:2693353     200    inhibitor
74994    598202  ESPORTSTMNT06:1720693     200       turret
74995    988622  ESPORTSTMNT01:3408793     200       turret
74996   1606464  ESPORTSTMNT03:3185926     100    inhibitor

[74997 rows x 4 columns]


In [34]:
first_inhib = first_buildings[first_buildings["buildingtype"] == "inhibitor"]
first_inhib = first_inhib.sort_values(by=['platformgameid', 'gametime'])
first_inhib["killerteamid_inhib"] = first_inhib["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_inhib.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
first_inhib.drop(columns=["buildingtype", "teamid"], inplace=True)
first_inhib.rename(columns={"gametime": "gametime_inhib"}, inplace=True)
display(first_inhib)

,gametime_inhib,platformgameid,killerteamid_inhib
70824,1708748,ESPORTSTMNT01:1110148,100
41143,1602302,ESPORTSTMNT01:1110162,100
62898,1546286,ESPORTSTMNT01:1110177,100
3623,1628728,ESPORTSTMNT01:1110187,100
50722,1694081,ESPORTSTMNT01:1110198,100
...,...,...,...
2262,1700113,LPL_B:330027,100
35688,1658796,LPL_B:330032,200
11354,1946801,LPL_B:330033,200
52312,1447915,LPL_B:330039,200


In [35]:
#first tower
first_tower = first_buildings[first_buildings["buildingtype"] == "turret"]
first_tower = first_tower.sort_values(by=['platformgameid', 'gametime'])
first_tower["killerteamid_tower"] = first_tower["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_tower.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
first_tower.drop(columns=["buildingtype", "teamid"], inplace=True)
first_inhib.rename(columns={"gametime": "gametime_tower"}, inplace=True)

In [36]:
buildings_joined = first_inhib.merge(first_tower, on="platformgameid")

In [37]:
first_kill = pd.read_csv("sql/first_blood.csv")
first_kill = first_kill.sort_values(by=['platformgameid', 'gametime'])
first_kill.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
display(first_kill)
first_kill.rename(columns={
    "gametime": "gametime_kill",
    "killerteamid": "killerteamid_kill",
}, inplace=True)

,gametime,platformgameid,killerteamid
4535,245612,ESPORTSTMNT01:1110148,100
9142,192537,ESPORTSTMNT01:1110162,100
21281,207700,ESPORTSTMNT01:1110177,100
35484,359245,ESPORTSTMNT01:1110187,200
11323,148483,ESPORTSTMNT01:1110198,100
...,...,...,...
21378,294277,LPL_B:330027,100
13430,227140,LPL_B:330032,100
44132,505054,LPL_B:330033,100
17859,302191,LPL_B:330039,100


In [41]:
buildings_with_kill = buildings_joined.merge(first_kill, on="platformgameid")

event_start_times = pd.read_csv("sql/event_start_times.csv")
buildings_with_kill = buildings_with_kill.merge(event_start_times, on="platformgameid")
display(buildings_with_kill)


,gametime_inhib,platformgameid,killerteamid_inhib,gametime,killerteamid_tower,gametime_kill,killerteamid_kill,eventtime
0,1708748,ESPORTSTMNT01:1110148,100,939441,100,245612,100,2019-05-01T09:11:39.980Z
1,1602302,ESPORTSTMNT01:1110162,100,677086,100,192537,100,2019-05-01T10:15:29.004Z
2,1546286,ESPORTSTMNT01:1110177,100,836608,100,207700,100,2019-05-01T11:15:13.609Z
3,1628728,ESPORTSTMNT01:1110187,100,626524,200,359245,200,2019-05-01T12:09:47.993Z
4,1694081,ESPORTSTMNT01:1110198,100,895424,100,148483,100,2019-05-01T13:01:30.206Z
...,...,...,...,...,...,...,...,...
25206,1700113,LPL_B:330027,100,885600,100,294277,100,2023-07-17T09:14:18.420Z
25207,1658796,LPL_B:330032,200,687897,200,227140,100,2023-07-17T10:03:31.166Z
25208,1946801,LPL_B:330033,200,900364,200,505054,100,2023-07-17T11:17:17.592Z
25209,1447915,LPL_B:330039,200,1008863,100,302191,100,2023-07-17T12:16:05.231Z


In [ ]:
buildings_with_kill.to_csv("joined.csv", index=False)